<a href="https://colab.research.google.com/github/ygopishetty/dolfin_fe/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import tensorflow as tf
import numpy as np
from nltk.stem import PorterStemmer

In [36]:
# User Intent
intents = [    {        "tag": "greeting",        "patterns": ["Hi there", "Hello", "Greetings", "Hi", "Hey"],
        "responses": ["Hello!", "Hi there! How can I help you?", "Hey! How can I assist you today?"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye"],
        "responses": ["Goodbye!", "See you soon!", "Take care!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thanks", "Thank you", "Thanks a lot"],
        "responses": ["You're welcome!", "No problem!", "Glad to help!"]
    },
    
    # Add more intents here 
]

In [37]:
# Extract data from intents
words = []
labels = []
docs_x = []
docs_y = []
for intent in intents:
    for pattern in intent['patterns']:
        # Tokenize each word in pattern
        w = pattern.split()
        words.extend(w)
        docs_x.append(w)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [38]:
# Stem and lower case words
stemmer = PorterStemmer()
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))


In [39]:
# Sort labels
labels = sorted(labels)


In [40]:
# Create training data
training = []
output = []
out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
    bag = []

    for w in words:
        if w in doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [41]:
# Convert training data to numpy arrays
training = np.array(training)
output = np.array(output)


In [42]:
# Define model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(training[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(output[0]), activation='softmax')
    ])

In [43]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
# Train model
model.fit(training, output, epochs=100, batch_size=8)

Epoch 1/100
2/2 [==============================] - 1s 7ms/step - loss: 1.0899 - accuracy: 0.3636
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 1.1278 - accuracy: 0.2727
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 1.0574 - accuracy: 0.6364
Epoch 4/100
2/2 [==============================] - 0s 7ms/step - loss: 1.0779 - accuracy: 0.4545
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 1.0345 - accuracy: 0.6364
Epoch 6/100
2/2 [==============================] - 0s 7ms/step - loss: 1.1168 - accuracy: 0.4545
Epoch 7/100
2/2 [==============================] - 0s 6ms/step - loss: 1.0274 - accuracy: 0.4545
Epoch 8/100
2/2 [==============================] - 0s 6ms/step - loss: 1.0410 - accuracy: 0.6364
Epoch 9/100
2/2 [==============================] - 0s 6ms/step - loss: 1.0305 - accuracy: 0.5455
Epoch 10/100
2/2 [==============================] - 0s 6ms/step - loss: 1.0916 - accuracy: 0.3636
Epoch 11/100
2/2 [===========

In [45]:
# Save model
model.save('chatbot_model.h5')

In [46]:
# Define function to preprocess user input
def preprocess_input(input):
    # Tokenize input
    input_words = input.split()
    # Stem and lower case words
    input_words = [stemmer.stem(w.lower()) for w in input_words]
    # Create bag of words
    input_bag = [0 for _ in range(len(words))]
    for w in input_words:
        if w in words:
            input_bag[words.index(w)] = 1
    # Return preprocessed input
    return np.array(input_bag)

In [47]:
def generate_response(input):
    # Preprocess input
    input_bag = preprocess_input(input)
    # Use model to predict output
    output = model.predict(np.array([input_bag]))[0]
    # Get index of highest probability
    index = np.argmax(output)
    # Get tag for predicted intent
    tag = labels[index]
    # Choose a random response from the chosen intent
    responses = [i['responses'] for i in intents if i['tag']==tag]
    if responses:
        response = np.random.choice(responses[0])
    else:
        response = "I'm sorry, I didn't understand that."
    return response

In [48]:
# Test chatbot
while True:
    input_text = input("You: ")
    if input_text.lower() == 'quit':
        break
    response = generate_response(input_text)
    print("Bot: " + response)

You: hi
1/1 [==============================] - 0s 68ms/step
Bot: Hello!


KeyboardInterrupt: ignored